 # 🔥 PyTorch Fundamentals Lab (ฉบับภาษาไทย)

 **วันที่ 1: จาก Tensor สู่ Training Loop**



 ยินดีต้อนรับสู่ PyTorch! บทเรียนนี้จะพาคุณจากศูนย์ไปสู่การสร้างและฝึก Neural Network



 ## 📚 สิ่งที่คุณจะได้เรียนรู้:

 - **Tensor** - โครงสร้างข้อมูลหลักของ PyTorch (เหมือน numpy array แต่ทรงพลังกว่า)

 - **Autograd** - ระบบคำนวณ gradient อัตโนมัติ (หัวใจของการเรียนรู้)

 - **Neural Networks** - การสร้างและฝึกโมเดล



 ## 📝 วิธีใช้งาน:

 - รันแต่ละ cell ตามลำดับ (Shift+Enter)

 - ทำแบบฝึกหัดที่มีเครื่องหมาย ✏️

 - ตรวจสอบความเข้าใจกับ ✅

 ---

 # 📦 Module 1: การติดตั้งและเตรียมสภาพแวดล้อม

 ---

 ก่อนเริ่มต้น เราต้องตั้งค่าสภาพแวดล้อมให้พร้อมก่อน

 ## 1.1 นำเข้า Libraries ที่จำเป็น



 **คำอธิบาย (ภาษาไทย):**

 - `torch` - Library หลักของ PyTorch

 - `torch.nn` - เก็บ building blocks สำหรับสร้าง neural network

 - `torch.optim` - เก็บ optimizer สำหรับปรับค่า weights

 - `numpy` - สำหรับการคำนวณทางคณิตศาสตร์

 - `matplotlib` - สำหรับสร้างกราฟ

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# แสดงเวอร์ชันของ PyTorch
print(f"📦 PyTorch version: {torch.__version__}")

# ตรวจสอบว่ามี GPU (CUDA) หรือไม่
print(f"🎮 CUDA available: {torch.cuda.is_available()}")


 ## 1.2 ตรวจสอบ Device ที่มี (CPU/GPU)



 **คำอธิบาย (ภาษาไทย):**



 PyTorch สามารถทำงานได้ทั้งบน CPU และ GPU:

 - **CPU** - ใช้ได้ทุกเครื่อง แต่ช้ากว่า

 - **GPU (CUDA)** - เร็วกว่ามากสำหรับ Deep Learning (ต้องมีการ์ดจอ NVIDIA)

 - **MPS** - สำหรับ Mac ที่ใช้ชิป Apple Silicon (M1/M2/M3)



 การเลือก device ที่เหมาะสมจะทำให้การ train เร็วขึ้นหลายเท่า!

In [ ]:
def get_device_info():
    """
    ฟังก์ชันแสดงข้อมูล device ที่มีอยู่
    (Function to display available device information)
    """
    print("=" * 50)
    print("📊 ข้อมูล DEVICE (DEVICE INFORMATION)")
    print("=" * 50)
    
    # CPU มีอยู่เสมอ
    print(f"\n📌 CPU: พร้อมใช้งานเสมอ (Always available)")
    
    # ตรวจสอบ NVIDIA GPU
    if torch.cuda.is_available():
        print(f"\n🚀 CUDA (NVIDIA GPU): ✅ พร้อมใช้งาน")
        print(f"   จำนวน GPU: {torch.cuda.device_count()}")
        
        for i in range(torch.cuda.device_count()):
            gpu_name = torch.cuda.get_device_name(i)
            total_mem = torch.cuda.get_device_properties(i).total_memory / 1e9
            print(f"   GPU {i}: {gpu_name}")
            print(f"         หน่วยความจำ: {total_mem:.2f} GB")
    else:
        print(f"\n⚠️  CUDA: ❌ ไม่พร้อมใช้งาน (จะใช้ CPU แทน)")
    
    # ตรวจสอบ Apple Silicon
    if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        print(f"\n🍎 MPS (Apple Silicon): ✅ พร้อมใช้งาน")
    
    print("=" * 50)

# เรียกใช้ฟังก์ชัน
get_device_info()


 ## 1.3 สร้างตัวแปร device สำหรับใช้ตลอดโปรแกรม



 **คำอธิบาย (ภาษาไทย):**



 เราจะสร้างตัวแปร `device` เพื่อใช้ตลอดทั้งโปรแกรม

 วิธีนี้ทำให้โค้ดของเราทำงานได้ทุกเครื่อง ไม่ว่าจะมี GPU หรือไม่



 **รูปแบบมาตรฐาน:**

 ```python

 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

 ```

In [ ]:
# สร้างตัวแปร device (ใช้ GPU ถ้ามี ไม่งั้นใช้ CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ กำลังใช้งาน device: {device}")


 ## 1.4 การย้าย Tensor ระหว่าง Devices



 **คำอธิบาย (ภาษาไทย):**



 เราสามารถย้าย tensor ระหว่าง CPU และ GPU ได้ด้วย `.to(device)`



 ⚠️ **ข้อควรจำ:** Tensor ที่อยู่คนละ device จะคำนวณร่วมกันไม่ได้!

 ต้องย้ายให้อยู่ device เดียวกันก่อน

In [ ]:
# สร้าง tensor บน CPU
x_cpu = torch.tensor([1, 2, 3])
print(f"🔵 x_cpu อยู่บน: {x_cpu.device}")

# ย้ายไปยัง device ที่เลือก (GPU หรือ CPU)
x_device = x_cpu.to(device)
print(f"🟢 x_device อยู่บน: {x_device.device}")


 ## 1.5 สร้าง Tensor บน Device ที่ต้องการโดยตรง



 **คำอธิบาย (ภาษาไทย):**



 แทนที่จะสร้างบน CPU แล้วค่อยย้าย เราสามารถสร้างบน device ที่ต้องการได้เลย

 วิธีนี้เร็วกว่าและประหยัดหน่วยความจำกว่า

In [ ]:
# สร้าง tensor บน device ที่ต้องการโดยตรง
y = torch.randn(3, 3, device=device)
print(f"🎯 y ถูกสร้างบน: {y.device}")
print(f"ค่าของ y:\n{y}")


 ## 1.6 การตั้งค่า Seed สำหรับ Reproducibility



 **คำอธิบาย (ภาษาไทย):**



 ในการทดลอง เราต้องการให้ผลลัพธ์เหมือนกันทุกครั้งที่รัน

 เรียกว่า **Reproducibility** (ความสามารถในการทำซ้ำ)



 วิธีการคือการตั้ง **seed** ให้ตัวสร้างเลขสุ่ม (random number generator)

 เมื่อ seed เดียวกัน → เลขสุ่มจะออกมาเหมือนกัน

In [ ]:
def set_seed(seed: int = 42):
    """
    ตั้งค่า seed สำหรับ reproducibility
    เรียกใช้ฟังก์ชันนี้ตอนเริ่มต้นการทดลอง!
    
    Args:
        seed: ค่า seed ที่ต้องการ (ค่าเริ่มต้นคือ 42)
    """
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # สำหรับ multi-GPU
        
        # ทำให้ CUDA operations เป็น deterministic
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    
    print(f"🌱 ตั้งค่า seed เป็น {seed} เรียบร้อย")

# เรียกใช้ฟังก์ชัน
set_seed(42)


 ## 1.7 ทดสอบ Reproducibility



 **คำอธิบาย (ภาษาไทย):**



 มาลองพิสูจน์ว่า seed ทำงานจริง

 เมื่อตั้ง seed เดียวกัน tensor สุ่มจะออกมาเหมือนกันทุกครั้ง

In [ ]:
# ทดสอบครั้งที่ 1
set_seed(42)
random_tensor_1 = torch.rand(3)
print(f"ครั้งที่ 1: {random_tensor_1}")

# ทดสอบครั้งที่ 2 (seed เดียวกัน)
set_seed(42)
random_tensor_2 = torch.rand(3)
print(f"ครั้งที่ 2: {random_tensor_2}")

# ตรวจสอบว่าเท่ากัน
print(f"\n✅ เท่ากันหรือไม่: {torch.equal(random_tensor_1, random_tensor_2)}")


 ## ✏️ แบบฝึกหัด 1.1: ฝึก Device Management



 **โจทย์:** สร้างฟังก์ชันที่:

 1. รับ tensor เป็น input

 2. แสดง device ปัจจุบันของ tensor

 3. ย้าย tensor ไปยัง device ที่ดีที่สุด (GPU ถ้ามี)

 4. คืนค่า tensor ที่ย้ายแล้ว

In [ ]:
def move_to_best_device(tensor):
    """
    ย้าย tensor ไปยัง device ที่ดีที่สุด
    
    Args:
        tensor: tensor ที่ต้องการย้าย
    
    Returns:
        tensor ที่ย้ายแล้ว
    """
    # เขียนโค้ดของคุณที่นี่ (YOUR CODE HERE)
    pass

# ทดสอบฟังก์ชันของคุณ:
# test_tensor = torch.ones(5)
# result = move_to_best_device(test_tensor)


 ---

 # 🧮 Module 2: Tensors - โครงสร้างข้อมูลหลักของ PyTorch

 ---



 **Tensor คืออะไร?**



 Tensor คือ array หลายมิติ (n-dimensional array) ที่มีความสามารถพิเศษ:

 - คำนวณบน GPU ได้

 - คำนวณ gradient อัตโนมัติได้

 - รองรับ broadcasting



 **เปรียบเทียบกับ NumPy:**

 | NumPy | PyTorch |

 |-------|---------|

 | ndarray | tensor |

 | np.array([1,2,3]) | torch.tensor([1,2,3]) |

 | np.zeros() | torch.zeros() |

 | np.dot() | torch.matmul() หรือ @ |

 ## 2.1 การสร้าง Tensor จาก Python List



 **คำอธิบาย (ภาษาไทย):**



 วิธีง่ายที่สุดในการสร้าง tensor คือการแปลงจาก Python list

In [ ]:
# สร้าง tensor 1 มิติ (vector)
a = torch.tensor([1, 2, 3])
print(f"Vector: {a}")
print(f"  Shape (รูปร่าง): {a.shape}")
print(f"  Dtype (ชนิดข้อมูล): {a.dtype}")


 ## 2.2 Tensor 2 มิติ (Matrix)



 **คำอธิบาย (ภาษาไทย):**



 Tensor 2 มิติ เรียกว่า Matrix หรือ ตาราง

 ใช้ nested list (list ซ้อน list) ในการสร้าง

In [ ]:
# สร้าง tensor 2 มิติ (matrix)
b = torch.tensor([
    [1, 2, 3],    # แถวที่ 1
    [4, 5, 6]     # แถวที่ 2
])
print(f"Matrix:\n{b}")
print(f"  Shape: {b.shape}")  # (2, 3) = 2 แถว, 3 คอลัมน์


 ## 2.3 การแปลง NumPy ↔ PyTorch



 **คำอธิบาย (ภาษาไทย):**



 PyTorch สามารถแปลงข้อมูลไปมาระหว่าง NumPy ได้ง่าย



 ⚠️ **ข้อควรระวัง:** `torch.from_numpy()` จะ **share memory** กับ NumPy array

 หมายความว่าถ้าแก้ไขอันหนึ่ง อีกอันจะเปลี่ยนตามด้วย!

In [ ]:
# NumPy → PyTorch
np_array = np.array([1.0, 2.0, 3.0])
tensor_from_np = torch.from_numpy(np_array)
print(f"NumPy array: {np_array}")
print(f"Tensor: {tensor_from_np}")


 ## 2.4 ทดสอบ Memory Sharing



 **คำอธิบาย (ภาษาไทย):**



 มาพิสูจน์ว่า memory ถูก share จริง

In [ ]:
# แก้ไข NumPy array
np_array[0] = 999
print(f"หลังแก้ไข NumPy: {np_array}")
print(f"Tensor เปลี่ยนตาม: {tensor_from_np}")

# คืนค่ากลับ
np_array[0] = 1.0


 ## 2.5 ฟังก์ชันสร้าง Tensor ที่ใช้บ่อย



 **คำอธิบาย (ภาษาไทย):**



 PyTorch มีฟังก์ชันสำเร็จรูปสำหรับสร้าง tensor แบบต่างๆ:



 | ฟังก์ชัน | ผลลัพธ์ |

 |---------|--------|

 | `torch.zeros()` | tensor ที่มีแต่ 0 |

 | `torch.ones()` | tensor ที่มีแต่ 1 |

 | `torch.rand()` | สุ่มจาก 0-1 (uniform) |

 | `torch.randn()` | สุ่มแบบ normal distribution |

 | `torch.arange()` | ลำดับตัวเลข |

 | `torch.eye()` | matrix หน่วย (identity) |

In [ ]:
# Zeros - สร้าง tensor ที่มีแต่ 0
zeros = torch.zeros(2, 3)  # 2 แถว, 3 คอลัมน์
print(f"Zeros (2x3):\n{zeros}")


In [ ]:
# Ones - สร้าง tensor ที่มีแต่ 1
ones = torch.ones(2, 3)
print(f"Ones (2x3):\n{ones}")


In [ ]:
# Rand - สุ่มจาก 0 ถึง 1 (uniform distribution)
random_uniform = torch.rand(2, 3)
print(f"Random Uniform (2x3):\n{random_uniform}")


In [ ]:
# Randn - สุ่มแบบ Normal Distribution (mean=0, std=1)
random_normal = torch.randn(2, 3)
print(f"Random Normal (2x3):\n{random_normal}")


In [ ]:
# Arange - ลำดับตัวเลข (เหมือน range ของ Python)
sequence = torch.arange(0, 10, 2)  # เริ่ม=0, จบ=10, step=2
print(f"Arange (0 ถึง 10, step=2): {sequence}")


In [ ]:
# Linspace - แบ่งช่วงเท่าๆ กัน
linspace = torch.linspace(0, 1, 5)  # แบ่ง 0-1 เป็น 5 จุด
print(f"Linspace (0 ถึง 1, 5 จุด): {linspace}")


In [ ]:
# Eye - Identity Matrix (matrix หน่วย)
identity = torch.eye(3)  # 3x3
print(f"Identity Matrix (3x3):\n{identity}")


 ## 2.6 Data Types (dtype) - ชนิดข้อมูล



 **คำอธิบาย (ภาษาไทย):**



 dtype กำหนดว่า tensor เก็บข้อมูลแบบไหน ส่งผลต่อ:

 - **ความแม่นยำ** (precision) - float64 แม่นกว่า float32

 - **หน่วยความจำ** - float32 ใช้น้อยกว่า float64

 - **ความเร็ว** - float32 คำนวณเร็วกว่า



 **dtype ที่ใช้บ่อย:**

 | dtype | ขนาด | ใช้เมื่อ |

 |-------|------|---------|

 | float32 | 4 bytes | ใช้ทั่วไป (ค่าเริ่มต้น) |

 | float64 | 8 bytes | ต้องการความแม่นยำสูง |

 | int64 | 8 bytes | ตัวเลขจำนวนเต็ม |

 | bool | 1 bit | True/False |

In [ ]:
# ค่าเริ่มต้นของ float คือ float32
x_default = torch.tensor([1.0])
print(f"Default float: {x_default.dtype}")  # torch.float32

# ค่าเริ่มต้นของ integer คือ int64
x_int = torch.tensor([1])
print(f"Default int: {x_int.dtype}")  # torch.int64


In [ ]:
# กำหนด dtype เอง
x_float32 = torch.tensor([1, 2, 3], dtype=torch.float32)
x_float64 = torch.tensor([1, 2, 3], dtype=torch.float64)

print(f"Float32: {x_float32.dtype}, ขนาด: {x_float32.element_size()} bytes ต่อตัว")
print(f"Float64: {x_float64.dtype}, ขนาด: {x_float64.element_size()} bytes ต่อตัว")


 ## 2.7 สร้าง Tensor ที่มีคุณสมบัติเหมือน Tensor อื่น



 **คำอธิบาย (ภาษาไทย):**



 บางครั้งเราต้องการสร้าง tensor ใหม่ที่มี dtype และ device เหมือน tensor ที่มีอยู่

 ใช้ฟังก์ชัน `*_like()`:

 - `torch.zeros_like(x)` - สร้าง zeros ที่มีคุณสมบัติเหมือน x

 - `torch.ones_like(x)` - สร้าง ones ที่มีคุณสมบัติเหมือน x

 - `torch.rand_like(x)` - สร้าง random ที่มีคุณสมบัติเหมือน x

In [ ]:
# สร้าง tensor ต้นแบบ
original = torch.randn(2, 3, dtype=torch.float64, device=device)
print(f"Original - device: {original.device}, dtype: {original.dtype}")
print(f"Original:\n{original}")

# สร้าง tensor ที่มีคุณสมบัติเหมือนกัน
zeros_like_original = torch.zeros_like(original)
print(f"\nZeros like - device: {zeros_like_original.device}, dtype: {zeros_like_original.dtype}")
print(f"Zeros like:\n{zeros_like_original}")


 ---

 ## 2.8 การเปลี่ยน Shape: reshape, view

 ---



 **คำอธิบาย (ภาษาไทย):**



 Shape คือ "รูปร่าง" ของ tensor บอกว่ามีกี่มิติ และแต่ละมิติมีขนาดเท่าไหร่



 **ตัวอย่าง:**

 - `(6,)` = 1 มิติ, 6 ตัว

 - `(2, 3)` = 2 มิติ, 2 แถว × 3 คอลัมน์

 - `(2, 2, 3)` = 3 มิติ, 2 × 2 × 3



 **กฎสำคัญ:** จำนวนข้อมูลทั้งหมดต้องเท่ากันหลัง reshape

 - `(6,)` → `(2, 3)` ได้ เพราะ 6 = 2×3

 - `(6,)` → `(2, 4)` ไม่ได้ เพราะ 6 ≠ 2×4

In [ ]:
# สร้าง tensor 1 มิติ
x = torch.arange(12)  # [0, 1, 2, ..., 11]
print(f"Original shape: {x.shape}")
print(f"Original: {x}")


 ### reshape เป็น 2 มิติ (Matrix)

In [ ]:
# Reshape เป็น 2D: 3 แถว × 4 คอลัมน์
x_2d = x.reshape(3, 4)
print(f"Shape หลัง reshape(3, 4): {x_2d.shape}")
print(f"Matrix:\n{x_2d}")


 ### reshape เป็น 3 มิติ

In [ ]:
# Reshape เป็น 3D: 2 "ก้อน" × 2 แถว × 3 คอลัมน์
x_3d = x.reshape(2, 2, 3)
print(f"Shape หลัง reshape(2, 2, 3): {x_3d.shape}")
print(f"3D Tensor:\n{x_3d}")


 ### ใช้ -1 ให้ PyTorch คำนวณมิติให้

In [ ]:
# ใช้ -1 = ให้ PyTorch คิดให้
x_auto = x.reshape(4, -1)  # 4 แถว, PyTorch คิดจำนวนคอลัมน์ให้
print(f"reshape(4, -1) → shape: {x_auto.shape}")  # (4, 3)


 ### ⚠️ ความแตกต่างระหว่าง view และ reshape



 **คำอธิบาย (ภาษาไทย):**



 - `view()` - คืนค่า **view** ของ tensor เดิม (share memory) ต้องเป็น contiguous

 - `reshape()` - อาจคืน view หรือ copy ก็ได้ ยืดหยุ่นกว่า



 ⚠️ **ระวัง:** ถ้าใช้ `view()` แล้วแก้ไขค่า tensor ต้นฉบับจะเปลี่ยนตาม!

In [ ]:
# ตัวอย่าง: view shares memory
x = torch.arange(6).reshape(2, 3)
print(f"x ก่อนแก้ไข:\n{x}")

y = x.view(3, 2)  # y คือ view ของ x (share memory)
y[0, 0] = 999     # แก้ไข y

print(f"\nx หลังแก้ไข y:\n{x}")  # x ก็เปลี่ยนด้วย!


 ## 2.9 squeeze และ unsqueeze - จัดการมิติขนาด 1



 **คำอธิบาย (ภาษาไทย):**



 - `squeeze()` - **ลบ** มิติที่มีขนาด 1 ออก

 - `unsqueeze()` - **เพิ่ม** มิติขนาด 1 เข้าไป



 **ทำไมต้องใช้?**

 โมเดลใน PyTorch มักต้องการ input shape แบบเฉพาะ เช่น:

 - ต้องการ batch dimension → ใช้ unsqueeze

 - ต้องลบมิติส่วนเกิน → ใช้ squeeze

In [ ]:
# ตัวอย่าง squeeze
x = torch.randn(1, 3, 1, 4, 1)
print(f"Shape เดิม: {x.shape}")  # (1, 3, 1, 4, 1)

# squeeze() ลบทุกมิติที่ขนาด = 1
squeezed = x.squeeze()
print(f"หลัง squeeze(): {squeezed.shape}")  # (3, 4)


In [ ]:
# squeeze เฉพาะมิติที่ต้องการ
x = torch.randn(1, 3, 1, 4, 1)
squeezed_0 = x.squeeze(0)  # ลบมิติที่ 0 เท่านั้น
print(f"หลัง squeeze(0): {squeezed_0.shape}")  # (3, 1, 4, 1)


In [ ]:
# ตัวอย่าง unsqueeze
x = torch.randn(3, 4)
print(f"Shape เดิม: {x.shape}")  # (3, 4)

# เพิ่มมิติที่ตำแหน่งต่างๆ
print(f"unsqueeze(0): {x.unsqueeze(0).shape}")   # (1, 3, 4) - เพิ่มที่ด้านหน้า
print(f"unsqueeze(1): {x.unsqueeze(1).shape}")   # (3, 1, 4) - เพิ่มตรงกลาง
print(f"unsqueeze(-1): {x.unsqueeze(-1).shape}") # (3, 4, 1) - เพิ่มที่ท้าย


 ### 🔑 Pattern สำคัญ: เพิ่ม Batch Dimension



 **คำอธิบาย (ภาษาไทย):**



 โมเดล Deep Learning มักต้องการ input ที่มี batch dimension

 เช่น รูปภาพ 1 รูป ต้องเป็น shape `(1, C, H, W)` ไม่ใช่ `(C, H, W)`

In [ ]:
# รูปภาพ 1 รูป: 3 channels, 224x224 pixels
single_image = torch.randn(3, 224, 224)
print(f"รูปเดี่ยว shape: {single_image.shape}")  # (3, 224, 224)

# เพิ่ม batch dimension
batched_image = single_image.unsqueeze(0)
print(f"หลังเพิ่ม batch: {batched_image.shape}")  # (1, 3, 224, 224)


 ## 2.10 Indexing และ Slicing



 **คำอธิบาย (ภาษาไทย):**



 การเข้าถึงข้อมูลใน tensor คล้ายกับ NumPy:

 - `x[i]` - เข้าถึงแถวที่ i

 - `x[i, j]` - เข้าถึง element ที่แถว i, คอลัมน์ j

 - `x[start:end]` - slice จาก start ถึง end-1

 - `x[:, j]` - ทุกแถว, คอลัมน์ j

 - `x[-1]` - แถวสุดท้าย

In [ ]:
# สร้าง matrix ตัวอย่าง
x = torch.arange(12).reshape(3, 4)
print(f"Matrix:\n{x}")
print(f"Shape: {x.shape}")


In [ ]:
# Basic indexing
print(f"x[1, 2] (แถว 1, คอลัมน์ 2): {x[1, 2]}")  # 6
print(f"x[0] (แถวแรก): {x[0]}")  # [0, 1, 2, 3]
print(f"x[-1] (แถวสุดท้าย): {x[-1]}")  # [8, 9, 10, 11]


In [ ]:
# Column indexing
print(f"x[:, 0] (คอลัมน์แรก): {x[:, 0]}")  # [0, 4, 8]
print(f"x[:, -1] (คอลัมน์สุดท้าย): {x[:, -1]}")  # [3, 7, 11]


In [ ]:
# Slicing
print(f"x[0:2, 1:3] (แถว 0-1, คอลัมน์ 1-2):\n{x[0:2, 1:3]}")
print(f"\nx[::2] (ทุกๆ 2 แถว):\n{x[::2]}")


 ## 2.11 Boolean Masking - การกรองด้วยเงื่อนไข



 **คำอธิบาย (ภาษาไทย):**



 เราสามารถใช้เงื่อนไข boolean เพื่อเลือกข้อมูลที่ต้องการ

 วิธีนี้ทรงพลังมากสำหรับการกรองและแก้ไขข้อมูล

In [ ]:
# สร้าง tensor ตัวอย่าง
x = torch.randn(3, 4)
print(f"Tensor:\n{x}")


In [ ]:
# สร้าง boolean mask
mask = x > 0
print(f"\nMask (ตำแหน่งที่ค่า > 0):\n{mask}")


In [ ]:
# ใช้ mask เลือกข้อมูล
positives = x[mask]
print(f"\nค่าที่ > 0: {positives}")


In [ ]:
# แก้ไขด้วย mask (ทำ ReLU ด้วยมือ!)
x[x < 0] = 0
print(f"\nหลังทำ x[x < 0] = 0 (ReLU):\n{x}")


 ## 2.12 torch.where - เลือกค่าตามเงื่อนไข



 **คำอธิบาย (ภาษาไทย):**



 `torch.where(condition, x, y)` คืนค่า:

 - ค่า x ถ้า condition เป็น True

 - ค่า y ถ้า condition เป็น False

In [ ]:
x = torch.tensor([-2., -1., 0., 1., 2.])
print(f"x: {x}")

# where(condition, if_true, if_false)
result = torch.where(x > 0, x, torch.zeros_like(x))
print(f"torch.where(x > 0, x, 0): {result}")


 ## ✏️ แบบฝึกหัด 2.1: ฝึก Tensor Manipulation



 **โจทย์:** จากตาราง 4×4 ให้:

 1. ดึงค่าบนเส้นทแยงมุม (diagonal)

 2. ดึง submatrix มุมบนขวา 2×2

 3. ดึงค่าทุกตัวที่มากกว่าค่าเฉลี่ย

In [ ]:
matrix = torch.arange(16).reshape(4, 4).float()
print(f"Matrix:\n{matrix}")

# เขียนโค้ดของคุณที่นี่ (YOUR CODE HERE)
# diagonal = ...
# upper_right = ...
# above_mean = ...


 ---

 ## 2.13 Broadcasting - การขยายมิติอัตโนมัติ

 ---



 **คำอธิบาย (ภาษาไทย):**



 Broadcasting ทำให้เราสามารถคำนวณระหว่าง tensor ที่มี shape ต่างกันได้



 **กฎ Broadcasting (พิจารณาจากขวาไปซ้าย):**

 1. มิติที่ขนาดเท่ากัน → ใช้ได้

 2. มิติที่ขนาด = 1 → ขยายได้

 3. มิติที่ไม่มี (ซ้าย) → ถือว่าเป็น 1



 **ตัวอย่าง:**

 - `(3, 4)` + `(4,)` → `(3, 4)` + `(1, 4)` → `(3, 4)` ✓

 - `(3, 4)` + `(3,)` → ไม่ได้ ✗

In [ ]:
# Broadcasting กับ scalar
x = torch.tensor([1, 2, 3])
print(f"x + 10 = {x + 10}")  # 10 ถูกขยายเป็น [10, 10, 10]


In [ ]:
# Broadcasting กับ vector
a = torch.ones(3, 4)       # shape: (3, 4)
b = torch.tensor([1, 2, 3, 4])  # shape: (4,) → (1, 4)

result = a + b
print(f"a shape: {a.shape}")
print(f"b shape: {b.shape}")
print(f"a + b shape: {result.shape}")
print(f"ผลลัพธ์:\n{result}")


 ### 🔑 การใช้ Broadcasting: Normalize ข้อมูล



 **คำอธิบาย (ภาษาไทย):**



 ตัวอย่างการใช้งานจริง: normalize แต่ละ column ให้มี mean=0, std=1

In [ ]:
# สร้างข้อมูล: 5 samples, 3 features
data = torch.randn(5, 3)
print(f"ข้อมูลดิบ:\n{data}")

# คำนวณ mean และ std แต่ละ column
mean = data.mean(dim=0, keepdim=True)  # shape: (1, 3)
std = data.std(dim=0, keepdim=True)    # shape: (1, 3)

# Normalize ด้วย broadcasting
normalized = (data - mean) / std

print(f"\nหลัง normalize:")
print(f"Mean แต่ละ column: {normalized.mean(dim=0)}")  # ควรใกล้ 0
print(f"Std แต่ละ column: {normalized.std(dim=0)}")    # ควรใกล้ 1


 ## 2.14 การคำนวณ Element-wise



 **คำอธิบาย (ภาษาไทย):**



 Element-wise operations คือการคำนวณทีละตัว (ตำแหน่งต่อตำแหน่ง)

In [ ]:
a = torch.tensor([1., 2., 3., 4.])
b = torch.tensor([2., 2., 2., 2.])

print("Element-wise Operations:")
print(f"  a + b = {a + b}")  # บวก
print(f"  a - b = {a - b}")  # ลบ
print(f"  a * b = {a * b}")  # คูณ (ไม่ใช่ matrix multiply!)
print(f"  a / b = {a / b}")  # หาร
print(f"  a ** b = {a ** b}")  # ยกกำลัง


In [ ]:
# ฟังก์ชันทางคณิตศาสตร์
print(f"exp(a) = {torch.exp(a)}")   # e^x
print(f"log(a) = {torch.log(a)}")   # ln(x)
print(f"sqrt(a) = {torch.sqrt(a)}") # รากที่สอง


 ## 2.15 Reduction Operations - การลดมิติ



 **คำอธิบาย (ภาษาไทย):**



 Reduction operations คือการรวมค่าหลายๆ ตัวให้เหลือค่าเดียว เช่น sum, mean, max



 **Parameter `dim`:** กำหนดว่าจะ reduce ตามมิติไหน

 - `dim=0` = reduce ตามแถว (ผลลัพธ์เหลือ 1 แถว)

 - `dim=1` = reduce ตามคอลัมน์ (ผลลัพธ์เหลือ 1 คอลัมน์)

In [ ]:
x = torch.tensor([
    [1., 2., 3.],
    [4., 5., 6.]
])
print(f"Tensor:\n{x}")


In [ ]:
# Sum ทั้งหมด
print(f"sum() รวมทั้งหมด: {x.sum()}")

# Sum ตาม dim
print(f"sum(dim=0) รวมตามแถว: {x.sum(dim=0)}")    # [1+4, 2+5, 3+6]
print(f"sum(dim=1) รวมตามคอลัมน์: {x.sum(dim=1)}")  # [1+2+3, 4+5+6]


In [ ]:
# Mean, Max, Argmax
print(f"mean() ค่าเฉลี่ย: {x.mean()}")
print(f"max() ค่าสูงสุด: {x.max()}")
print(f"argmax() ตำแหน่งค่าสูงสุด: {x.argmax()}")  # ตำแหน่งใน flattened array


 ### keepdim=True - รักษามิติไว้



 **คำอธิบาย (ภาษาไทย):**



 `keepdim=True` ทำให้ผลลัพธ์ยังคงมิติเดิม (แต่ขนาด = 1)

 มีประโยชน์สำหรับ broadcasting ในขั้นตอนถัดไป

In [ ]:
print(f"ไม่มี keepdim: {x.sum(dim=1).shape}")        # (2,)
print(f"มี keepdim=True: {x.sum(dim=1, keepdim=True).shape}")  # (2, 1)


 ## 2.16 ⭐ Matrix Multiplication - การคูณเมทริกซ์



 **คำอธิบาย (ภาษาไทย):**



 Matrix multiplication คือหัวใจของ Neural Networks!



 **กฎ:** `(m, n) @ (n, p)` → `(m, p)`

 - มิติด้านในต้องเท่ากัน (n)

 - ผลลัพธ์มีขนาด (มิติซ้ายของตัวแรก, มิติขวาของตัวหลัง)



 **วิธีเขียน (3 วิธี เหมือนกันหมด):**

 - `A @ B` - แนะนำ! อ่านง่าย

 - `torch.matmul(A, B)` - ชัดเจน

 - `torch.mm(A, B)` - สำหรับ 2D เท่านั้น

In [ ]:
A = torch.randn(2, 3)  # 2 แถว, 3 คอลัมน์
B = torch.randn(3, 4)  # 3 แถว, 4 คอลัมน์

# Matrix multiplication
C = A @ B  # (2, 3) @ (3, 4) → (2, 4)

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"A @ B shape: {C.shape}")


 ### Batch Matrix Multiplication



 **คำอธิบาย (ภาษาไทย):**



 เมื่อมีหลาย matrix ที่ต้องคูณพร้อมกัน (เช่น batch ของข้อมูล)

In [ ]:
# 32 คู่ของ matrix ที่ต้องคูณกัน
batch_A = torch.randn(32, 4, 5)  # 32 matrices ขนาด 4×5
batch_B = torch.randn(32, 5, 3)  # 32 matrices ขนาด 5×3

batch_C = batch_A @ batch_B  # 32 matrices ขนาด 4×3
print(f"Batch matmul: {batch_A.shape} @ {batch_B.shape} = {batch_C.shape}")


 ## 2.17 Concatenate และ Stack



 **คำอธิบาย (ภาษาไทย):**



 - `torch.cat()` - ต่อ tensor ตามมิติที่กำหนด (ไม่เพิ่มมิติใหม่)

 - `torch.stack()` - ซ้อน tensor เป็นมิติใหม่

In [ ]:
a = torch.ones(2, 3)
b = torch.zeros(2, 3)

# Concatenate ตาม dim=0 (ต่อแนวตั้ง)
cat_dim0 = torch.cat([a, b], dim=0)
print(f"cat(dim=0): {cat_dim0.shape}")  # (4, 3)
print(f"ผลลัพธ์:\n{cat_dim0}")


In [ ]:
# Concatenate ตาม dim=1 (ต่อแนวนอน)
cat_dim1 = torch.cat([a, b], dim=1)
print(f"cat(dim=1): {cat_dim1.shape}")  # (2, 6)
print(f"ผลลัพธ์:\n{cat_dim1}")


In [ ]:
# Stack (สร้างมิติใหม่)
stacked = torch.stack([a, b], dim=0)
print(f"stack(dim=0): {stacked.shape}")  # (2, 2, 3)
print(f"ผลลัพธ์:\n{stacked}")


 ## ✏️ แบบฝึกหัด 2.2: Implement Functions



 **โจทย์ A:** Implement Softmax



 สูตร: `softmax(x) = exp(x) / sum(exp(x))`



 **เคล็ดลับ:** ลบค่า max ก่อน exp เพื่อป้องกัน numerical overflow

In [ ]:
def softmax(x, dim=-1):
    """
    คำนวณ softmax ตามมิติที่กำหนด
    
    Args:
        x: input tensor
        dim: มิติที่จะคำนวณ softmax (default=-1 คือมิติสุดท้าย)
    
    Returns:
        tensor ที่ผ่าน softmax แล้ว (ผลรวมตาม dim = 1)
    """
    # เขียนโค้ดของคุณที่นี่ (YOUR CODE HERE)
    pass

# ทดสอบ:
# x = torch.tensor([[1., 2., 3.], [1., 1., 1.]])
# print(softmax(x))  # แต่ละแถวควรรวมเป็น 1


 **โจทย์ B:** Implement Batch Cosine Similarity



 สูตร: `cos_sim(a, b) = (a · b) / (||a|| × ||b||)`

In [ ]:
def batch_cosine_similarity(a, b):
    """
    คำนวณ cosine similarity ระหว่าง vector คู่ที่ตรงกัน
    
    Args:
        a: tensor shape (batch, features)
        b: tensor shape (batch, features)
    
    Returns:
        tensor shape (batch,) ค่า similarity แต่ละคู่
    """
    # เขียนโค้ดของคุณที่นี่ (YOUR CODE HERE)
    pass

# ทดสอบ:
# a = torch.tensor([[1., 0., 0.], [1., 1., 0.]])
# b = torch.tensor([[1., 0., 0.], [0., 1., 0.]])
# print(batch_cosine_similarity(a, b))  # ควรได้ [1.0, 0.707...]


 ---

 # 🎯 Module 3: Autograd - หัวใจของการเรียนรู้

 ---



 **Autograd คืออะไร?**



 Autograd (Automatic Differentiation) คือระบบที่คำนวณ gradient อัตโนมัติ

 เป็นหัวใจสำคัญที่ทำให้ Deep Learning เป็นไปได้!



 **ทำไมต้องใช้ Gradient?**



 การ train neural network ใช้ Gradient Descent:

 1. คำนวณ prediction

 2. คำนวณ loss (error)

 3. คำนวณ gradient ของ loss ต่อ parameters

 4. ปรับ parameters: `param = param - learning_rate × gradient`



 Autograd ช่วยในขั้นตอนที่ 3 โดยอัตโนมัติ!

 ## 3.1 requires_grad - เปิดใช้งาน Gradient Tracking



 **คำอธิบาย (ภาษาไทย):**



 โดยปกติ tensor จะไม่ติดตาม gradient (ประหยัดหน่วยความจำ)

 ต้องเปิดด้วย `requires_grad=True` เพื่อให้ autograd ทำงาน

In [ ]:
# ปกติ tensor ไม่ track gradient
x = torch.tensor([1., 2., 3.])
print(f"requires_grad (default): {x.requires_grad}")  # False


In [ ]:
# เปิด gradient tracking
x = torch.tensor([1., 2., 3.], requires_grad=True)
print(f"requires_grad (enabled): {x.requires_grad}")  # True


 ## 3.2 Computation Graph - กราฟการคำนวณ



 **คำอธิบาย (ภาษาไทย):**



 เมื่อเราทำ operation กับ tensor ที่มี `requires_grad=True`

 PyTorch จะสร้าง "computation graph" เก็บไว้



 กราฟนี้บันทึกว่า:

 - ค่าแต่ละตัวมาจาก operation อะไร

 - ต้องคำนวณ gradient อย่างไร

In [ ]:
x = torch.tensor([2.0], requires_grad=True)

# Forward pass: y = x² + 3x + 1
y = x ** 2 + 3 * x + 1

print(f"x = {x.item()}")
print(f"y = x² + 3x + 1 = {y.item()}")
print(f"y.grad_fn = {y.grad_fn}")  # แสดง operation ที่สร้าง y


 ## 3.3 backward() - คำนวณ Gradient



 **คำอธิบาย (ภาษาไทย):**



 เมื่อเรียก `.backward()` PyTorch จะคำนวณ gradient อัตโนมัติ

 ผลลัพธ์เก็บไว้ใน `.grad` ของแต่ละ tensor

In [ ]:
x = torch.tensor([2.0], requires_grad=True)
y = x ** 2 + 3 * x + 1

# คำนวณ gradient
y.backward()

# ผลลัพธ์เก็บใน x.grad
print(f"x = {x.item()}")
print(f"y = {y.item()}")
print(f"dy/dx = {x.grad.item()}")

# ตรวจสอบ: dy/dx = 2x + 3 = 2(2) + 3 = 7 ✓


 ## 3.4 Gradient สำหรับ Vector Output



 **คำอธิบาย (ภาษาไทย):**



 ถ้า output เป็น vector (ไม่ใช่ scalar) ต้องระบุ "upstream gradient"

 ในทางปฏิบัติ เรามักจะ reduce เป็น scalar ก่อน (เช่น loss function)

In [ ]:
# กรณี output เป็น vector
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x ** 2  # y เป็น vector [1, 4, 9]

# ต้องระบุ upstream gradient
upstream = torch.tensor([1., 1., 1.])
y.backward(upstream)

print(f"x = {x}")
print(f"y = x² = {y}")
print(f"dy/dx = 2x = {x.grad}")


 ## 3.5 Pattern ที่ใช้บ่อย: Reduce เป็น Scalar



 **คำอธิบาย (ภาษาไทย):**



 ในการ train จริง loss จะเป็น scalar เสมอ

 ทำให้ไม่ต้องระบุ upstream gradient

In [ ]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = (x ** 2).sum()  # Reduce เป็น scalar: 1 + 4 + 9 = 14

y.backward()  # ไม่ต้องระบุ argument

print(f"y = sum(x²) = {y.item()}")
print(f"dy/dx = 2x = {x.grad}")


 ## 3.6 ⚠️ Gradient Accumulation - ปัญหาที่พบบ่อย



 **คำอธิบาย (ภาษาไทย):**



 **สำคัญมาก!** Gradient จะ **สะสม** (accumulate) ทุกครั้งที่เรียก backward()

 ถ้าไม่ clear จะทำให้ค่า gradient ผิด!

In [ ]:
x = torch.tensor([1., 2., 3.], requires_grad=True)

# backward ครั้งที่ 1
y1 = (x ** 2).sum()
y1.backward()
print(f"หลัง backward ครั้งที่ 1: {x.grad}")  # [2, 4, 6]

# backward ครั้งที่ 2 - gradient สะสม!
y2 = (x ** 3).sum()
y2.backward()
print(f"หลัง backward ครั้งที่ 2: {x.grad}")  # [2+3, 4+12, 6+27] = [5, 16, 33]


 ## 3.7 วิธีแก้: Zero Gradients ก่อน backward



 **คำอธิบาย (ภาษาไทย):**



 ต้อง clear gradient ก่อนทุกครั้งที่จะ backward ใหม่

In [ ]:
x = torch.tensor([1., 2., 3.], requires_grad=True)

# backward ครั้งที่ 1
y1 = (x ** 2).sum()
y1.backward()
print(f"Gradient 1: {x.grad}")

# 🔑 Clear gradient!
x.grad.zero_()

# backward ครั้งที่ 2
y2 = (x ** 3).sum()
y2.backward()
print(f"Gradient 2 (หลัง clear): {x.grad}")  # ตอนนี้ถูกต้องแล้ว


 ## 3.8 torch.no_grad() - ปิด Gradient Tracking



 **คำอธิบาย (ภาษาไทย):**



 ตอน inference (ทดสอบโมเดล) เราไม่ต้องการ gradient

 การปิด gradient ทำให้:

 - **เร็วขึ้น** - ไม่ต้องสร้าง computation graph

 - **ประหยัดหน่วยความจำ** - ไม่ต้องเก็บ intermediate values

In [ ]:
x = torch.tensor([1., 2., 3.], requires_grad=True)

# ปกติ: มี gradient tracking
y = x ** 2
print(f"ปกติ - requires_grad: {y.requires_grad}")

# ใน no_grad: ไม่มี gradient tracking
with torch.no_grad():
    z = x ** 2
    print(f"ใน no_grad - requires_grad: {z.requires_grad}")


 ## 3.9 Pattern: Evaluation Mode



 **คำอธิบาย (ภาษาไทย):**



 รูปแบบมาตรฐานในการ evaluate โมเดล:

In [ ]:
def evaluate(model, data):
    """
    ประเมินผลโมเดลอย่างถูกต้อง
    """
    model.eval()           # ตั้งเป็น evaluation mode
    with torch.no_grad():  # ปิด gradient
        outputs = model(data)
    return outputs


 ## 3.10 detach() - ตัด Tensor ออกจาก Graph



 **คำอธิบาย (ภาษาไทย):**



 `detach()` สร้าง tensor ใหม่ที่ไม่เชื่อมกับ computation graph

 มีประโยชน์เมื่อต้องการใช้ค่าโดยไม่ให้ gradient ไหลผ่าน

In [ ]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x ** 2

# detach สร้าง tensor ใหม่ที่ไม่ต้องการ grad
z = y.detach()

print(f"y.requires_grad: {y.requires_grad}")
print(f"z.requires_grad: {z.requires_grad}")


 ## 3.11 Gradient Clipping - ป้องกัน Exploding Gradients



 **คำอธิบาย (ภาษาไทย):**



 บางครั้ง gradient มีค่าสูงมาก (exploding gradients)

 ทำให้การ train ไม่เสถียร



 **วิธีแก้:** จำกัดขนาดของ gradient

 - `clip_grad_norm_()` - จำกัด L2 norm รวม (นิยมใช้)

 - `clip_grad_value_()` - จำกัดค่าแต่ละตัว

In [ ]:
# สร้าง parameters ที่มี gradient สูง
params = torch.tensor([1., 2., 3.], requires_grad=True)
loss = (params ** 4).sum()  # Gradient จะสูงมาก
loss.backward()

print(f"Gradient เดิม: {params.grad}")
print(f"Gradient norm: {params.grad.norm():.4f}")


 ### Clip by Norm (แนะนำ)

In [ ]:
params = torch.tensor([1., 2., 3.], requires_grad=True)
loss = (params ** 4).sum()
loss.backward()

max_norm = 1.0
torch.nn.utils.clip_grad_norm_([params], max_norm)

print(f"หลัง clip by norm (max=1.0):")
print(f"  Gradient: {params.grad}")
print(f"  Norm: {params.grad.norm():.4f}")


 ## ✏️ แบบฝึกหัด 3.1: ตรวจสอบ Gradient ด้วยมือ



 **โจทย์:** ตรวจสอบว่า autograd คำนวณ gradient ถูกต้อง



 ให้ `g(x) = sin(x) × exp(-x)` ที่ x = 1



 คำนวณ gradient ด้วย autograd แล้วเทียบกับ analytical solution

In [ ]:
# ตัวอย่าง: f(x) = x³ - 2x² + x
def f(x):
    return x**3 - 2*x**2 + x

def df_dx_analytical(x):
    """Analytical derivative: f'(x) = 3x² - 4x + 1"""
    return 3*x**2 - 4*x + 1

# ทดสอบที่ x = 2
x = torch.tensor([2.0], requires_grad=True)
y = f(x)
y.backward()

print(f"Autograd df/dx ที่ x=2: {x.grad.item()}")
print(f"Analytical df/dx ที่ x=2: {df_dx_analytical(2.0)}")


In [ ]:
# โจทย์ของคุณ: g(x) = sin(x) × exp(-x) ที่ x = 1
# เขียนโค้ดของคุณที่นี่ (YOUR CODE HERE)


 ---

 # 🏗️ Module 4: Neural Network Building Blocks

 ---



 ตอนนี้เรามาสร้าง Neural Network จริงๆ กัน!

 ## 4.1 nn.Module - คลาสพื้นฐานของ Neural Networks



 **คำอธิบาย (ภาษาไทย):**



 `nn.Module` คือ base class ของทุก neural network ใน PyTorch



 **โครงสร้างมาตรฐาน:**

 1. `__init__()` - กำหนด layers

 2. `forward()` - กำหนดการไหลของข้อมูล

In [ ]:
class SimpleNet(nn.Module):
    """
    Neural Network อย่างง่าย
    
    โครงสร้าง:
    Input → Linear → ReLU → Linear → Output
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()  # สำคัญ! ต้องเรียก parent __init__
        
        # กำหนด layers
        self.fc1 = nn.Linear(input_size, hidden_size)   # Fully Connected 1
        self.fc2 = nn.Linear(hidden_size, output_size)  # Fully Connected 2
        self.relu = nn.ReLU()                           # Activation
    
    def forward(self, x):
        """กำหนดการไหลของข้อมูล"""
        x = self.fc1(x)   # Linear: y = Wx + b
        x = self.relu(x)  # Activation: max(0, x)
        x = self.fc2(x)   # Linear อีกครั้ง
        return x

# สร้าง model
model = SimpleNet(input_size=10, hidden_size=32, output_size=5)
print(model)


 ## 4.2 ดู Parameters ของ Model



 **คำอธิบาย (ภาษาไทย):**



 Model มี parameters (weights และ biases) ที่จะถูกเรียนรู้

In [ ]:
print("📊 Model Parameters:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape}")

# นับจำนวน parameters ทั้งหมด
total_params = sum(p.numel() for p in model.parameters())
print(f"\n📈 Total parameters: {total_params:,}")


 ## 4.3 Forward Pass - ส่งข้อมูลผ่าน Model



 **คำอธิบาย (ภาษาไทย):**



 เพียงแค่ "เรียก" model เหมือนฟังก์ชัน

 PyTorch จะเรียก `forward()` ให้อัตโนมัติ

In [ ]:
# สร้าง input: batch=4, features=10
x = torch.randn(4, 10)

# Forward pass (เรียก model เหมือนฟังก์ชัน)
output = model(x)

print(f"Input shape: {x.shape}")     # (4, 10)
print(f"Output shape: {output.shape}") # (4, 5)


 ## 4.4 ย้าย Model ไป GPU



 **คำอธิบาย (ภาษาไทย):**



 ใช้ `.to(device)` เหมือน tensor

 จะย้าย parameters ทั้งหมดไป device ที่กำหนด

In [ ]:
model = model.to(device)
print(f"Model อยู่บน: {next(model.parameters()).device}")


 ---

 ## 4.5 Common Layers - Layers ที่ใช้บ่อย

 ---

 ### Linear Layer (Fully Connected)



 **คำอธิบาย (ภาษาไทย):**



 `nn.Linear(in_features, out_features)` ทำการคำนวณ `y = xW^T + b`

 - `W` shape: (out_features, in_features)

 - `b` shape: (out_features,)

In [ ]:
linear = nn.Linear(in_features=10, out_features=5)

print(f"Linear layer: {linear}")
print(f"  Weight shape: {linear.weight.shape}")  # (5, 10)
print(f"  Bias shape: {linear.bias.shape}")      # (5,)

# ทดสอบ
x = torch.randn(3, 10)  # batch=3
y = linear(x)
print(f"\n  Input: {x.shape} → Output: {y.shape}")


 ### Embedding Layer



 **คำอธิบาย (ภาษาไทย):**



 `nn.Embedding` แปลง integer indices เป็น dense vectors

 ใช้สำหรับ NLP (แปลงคำเป็น vector) หรือ categorical data

In [ ]:
vocab_size = 1000       # จำนวนคำทั้งหมด
embedding_dim = 64      # ขนาดของ embedding vector

embedding = nn.Embedding(vocab_size, embedding_dim)

# Input: indices ของคำ (integers)
word_indices = torch.tensor([1, 45, 999, 0, 123])
embedded = embedding(word_indices)

print(f"Embedding: {vocab_size} คำ → {embedding_dim} มิติ")
print(f"Input: {word_indices.shape} → Output: {embedded.shape}")


 ### Conv2d (Preview)



 **คำอธิบาย (ภาษาไทย):**



 Convolutional layer สำหรับ image processing

 จะเรียนรู้ลึกขึ้นใน lab ถัดไป

In [ ]:
conv = nn.Conv2d(
    in_channels=3,      # RGB = 3 channels
    out_channels=16,    # สร้าง 16 feature maps
    kernel_size=3,      # ขนาด filter 3×3
    padding=1           # เพิ่ม padding เพื่อรักษาขนาด
)

# Input: batch=1, channels=3, height=32, width=32
image = torch.randn(1, 3, 32, 32)
features = conv(image)

print(f"Conv2d: 3 channels → 16 features")
print(f"Input: {image.shape} → Output: {features.shape}")


 ---

 ## 4.6 Activation Functions - ฟังก์ชันกระตุ้น

 ---



 **คำอธิบาย (ภาษาไทย):**



 Activation functions สร้าง non-linearity ให้ neural network

 ถ้าไม่มี activation function, network หลายชั้นก็ไม่ต่างจากชั้นเดียว!

 ### ReLU - Rectified Linear Unit



 สูตร: `max(0, x)`

 - ถ้า x > 0 → คืน x

 - ถ้า x ≤ 0 → คืน 0



 **ข้อดี:** ง่าย, เร็ว, ใช้กันแพร่หลาย

In [ ]:
relu = nn.ReLU()
x = torch.tensor([-2., -1., 0., 1., 2.])
print(f"Input: {x}")
print(f"ReLU: {relu(x)}")


 ### Sigmoid



 สูตร: `1 / (1 + e^(-x))`

 - Output อยู่ในช่วง (0, 1)

 - ใช้สำหรับ probability, binary classification

In [ ]:
sigmoid = nn.Sigmoid()
x = torch.tensor([-2., -1., 0., 1., 2.])
print(f"Input: {x}")
print(f"Sigmoid: {sigmoid(x)}")


 ### Tanh



 สูตร: `(e^x - e^(-x)) / (e^x + e^(-x))`

 - Output อยู่ในช่วง (-1, 1)

In [ ]:
tanh = nn.Tanh()
x = torch.tensor([-2., -1., 0., 1., 2.])
print(f"Input: {x}")
print(f"Tanh: {tanh(x)}")


 ### Visualize Activation Functions

In [ ]:
x = torch.linspace(-5, 5, 100)

fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# ReLU
axes[0, 0].plot(x, nn.ReLU()(x))
axes[0, 0].set_title("ReLU: max(0, x)")
axes[0, 0].grid(True)
axes[0, 0].axhline(y=0, color='k', linewidth=0.5)
axes[0, 0].axvline(x=0, color='k', linewidth=0.5)

# Sigmoid
axes[0, 1].plot(x, nn.Sigmoid()(x))
axes[0, 1].set_title("Sigmoid: 1/(1+e^-x)")
axes[0, 1].grid(True)
axes[0, 1].axhline(y=0.5, color='r', linestyle='--', linewidth=0.5)

# Tanh
axes[1, 0].plot(x, nn.Tanh()(x))
axes[1, 0].set_title("Tanh: (e^x - e^-x)/(e^x + e^-x)")
axes[1, 0].grid(True)
axes[1, 0].axhline(y=0, color='k', linewidth=0.5)

# GELU (ใช้ใน Transformers)
axes[1, 1].plot(x, nn.GELU()(x))
axes[1, 1].set_title("GELU")
axes[1, 1].grid(True)
axes[1, 1].axhline(y=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()


 ### Softmax - แปลงเป็น Probability Distribution



 **คำอธิบาย (ภาษาไทย):**



 Softmax แปลง logits (raw scores) เป็น probabilities ที่รวมกันได้ 1

In [ ]:
logits = torch.tensor([[2.0, 1.0, 0.1]])

softmax = nn.Softmax(dim=1)
probs = softmax(logits)

print(f"Logits: {logits}")
print(f"Probabilities: {probs}")
print(f"ผลรวม: {probs.sum().item()}")  # ต้องเป็น 1


 ---

 ## 4.7 Loss Functions - ฟังก์ชันวัดความผิดพลาด

 ---



 **คำอธิบาย (ภาษาไทย):**



 Loss function วัดว่า prediction ของเราห่างจาก target มากแค่ไหน

 เป้าหมายของการ train คือทำให้ loss ต่ำที่สุด

 ### MSE Loss - สำหรับ Regression



 สูตร: `mean((prediction - target)²)`

In [ ]:
predictions = torch.tensor([2.5, 0.0, 2.0])
targets = torch.tensor([3.0, 0.0, 2.0])

mse_loss = nn.MSELoss()
loss = mse_loss(predictions, targets)

print(f"Predictions: {predictions}")
print(f"Targets: {targets}")
print(f"MSE Loss: {loss.item():.4f}")

# คำนวณด้วยมือ
manual_mse = ((predictions - targets) ** 2).mean()
print(f"คำนวณด้วยมือ: {manual_mse.item():.4f}")


 ### Cross Entropy Loss - สำหรับ Multi-class Classification



 **คำอธิบาย (ภาษาไทย):**



 ใช้สำหรับ classification ที่มีหลาย class



 ⚠️ **สำคัญ:** รับ **logits** (raw scores) ไม่ใช่ softmax probabilities!

In [ ]:
# Logits = raw scores ก่อน softmax
logits = torch.tensor([
    [2.0, 1.0, 0.1],   # Sample 1: สูงสุดที่ class 0
    [0.1, 0.1, 2.0]    # Sample 2: สูงสุดที่ class 2
])

# Target class indices
targets = torch.tensor([0, 2])  # Sample 1 = class 0, Sample 2 = class 2

ce_loss = nn.CrossEntropyLoss()
loss = ce_loss(logits, targets)

print(f"Logits:\n{logits}")
print(f"Targets: {targets}")
print(f"Cross Entropy Loss: {loss.item():.4f}")


 ### BCE Loss - สำหรับ Binary Classification



 **คำอธิบาย (ภาษาไทย):**



 ใช้สำหรับ classification 2 class หรือ multi-label classification

In [ ]:
logits = torch.tensor([0.5, -1.0, 2.0])   # Raw scores
targets = torch.tensor([1.0, 0.0, 1.0])   # Binary labels

bce_loss = nn.BCEWithLogitsLoss()  # รวม Sigmoid + BCELoss
loss = bce_loss(logits, targets)

print(f"Logits: {logits}")
print(f"Targets: {targets}")
print(f"BCE with Logits Loss: {loss.item():.4f}")


 ### 🔑 สรุป: เลือก Loss Function อย่างไร



 | Task | Loss Function | Output ที่รับ |

 |------|--------------|--------------|

 | Regression | MSELoss | ค่าใดก็ได้ |

 | Multi-class (1 label) | CrossEntropyLoss | Logits |

 | Binary classification | BCEWithLogitsLoss | Logit |

 | Multi-label | BCEWithLogitsLoss | Logits |

 ---

 ## 4.8 Optimizers - ตัวปรับค่า Parameters

 ---



 **คำอธิบาย (ภาษาไทย):**



 Optimizer ใช้ gradient เพื่อปรับค่า parameters ของ model

 สูตรพื้นฐาน: `new_param = old_param - learning_rate × gradient`

In [ ]:
# สร้าง model อย่างง่าย
model = nn.Linear(10, 2)

# Optimizers ต่างๆ
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01)
optimizer_adam = optim.Adam(model.parameters(), lr=0.001)
optimizer_adamw = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

print("Optimizers ที่นิยม:")
print("• SGD - Stochastic Gradient Descent (พื้นฐาน)")
print("• Adam - Adaptive Moment Estimation (นิยมที่สุด)")
print("• AdamW - Adam + Weight Decay (ดีสำหรับ Transformers)")


 ### Learning Rate คืออะไร?



 **คำอธิบาย (ภาษาไทย):**



 Learning rate (lr) ควบคุมขนาดของการ update

 - lr สูง → update เยอะ → เรียนรู้เร็ว แต่อาจ overshoot

 - lr ต่ำ → update น้อย → เรียนรู้ช้า แต่เสถียรกว่า

In [ ]:
# แสดงผลกระทบของ learning rate
model = nn.Linear(10, 1)
original_weight = model.weight.data.clone()

# สร้าง gradient
x = torch.randn(1, 10)
y = torch.tensor([[1.0]])
loss = (model(x) - y) ** 2
loss.backward()

print(f"Gradient magnitude: {model.weight.grad.abs().mean():.4f}")
print(f"\nEffect of different learning rates:")

for lr in [0.001, 0.01, 0.1]:
    model.weight.data = original_weight.clone()
    model.weight.data -= lr * model.weight.grad
    change = (model.weight.data - original_weight).abs().mean()
    print(f"  lr={lr}: weight changed by {change:.6f}")


 ## 4.9 🔑 Training Loop - Pattern ที่ใช้ทุกครั้ง



 **คำอธิบาย (ภาษาไทย):**



 นี่คือ pattern มาตรฐานที่ใช้ในการ train model ทุกครั้ง:



 ```python

 for epoch in range(n_epochs):

     model.train()              # 1. ตั้ง training mode

     outputs = model(x)         # 2. Forward pass

     loss = criterion(outputs, y)  # 3. คำนวณ loss



     optimizer.zero_grad()      # 4. Clear gradients

     loss.backward()            # 5. Compute gradients

     optimizer.step()           # 6. Update parameters

 ```

In [ ]:
# ตัวอย่าง Training Step
model = SimpleNet(10, 32, 5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def training_step(model, x, y, criterion, optimizer):
    """
    Training step มาตรฐาน
    
    Args:
        model: neural network
        x: input data
        y: target labels
        criterion: loss function
        optimizer: optimizer
    
    Returns:
        loss value
    """
    model.train()  # 1. Training mode
    
    # 2. Forward pass
    outputs = model(x)
    
    # 3. คำนวณ loss
    loss = criterion(outputs, y)
    
    # 4-6. Backward pass
    optimizer.zero_grad()  # Clear gradients
    loss.backward()        # Compute gradients
    optimizer.step()       # Update parameters
    
    return loss.item()

# ทดสอบ
x = torch.randn(4, 10).to(device)
y = torch.randint(0, 5, (4,)).to(device)
loss = training_step(model, x, y, criterion, optimizer)
print(f"Training loss: {loss:.4f}")


 ---

 # 🧪 Lab A: Linear Regression จากศูนย์

 ---



 มาลองสร้าง Linear Regression โดยใช้ tensor operations และ autograd โดยตรง!

 ## Step 1: สร้างข้อมูล



 เราจะสร้างข้อมูลที่มีความสัมพันธ์: `y = 3x + 2 + noise`

In [ ]:
set_seed(42)

# สร้างข้อมูล
n_samples = 100
X = torch.randn(n_samples, 1)
y_true = 3 * X + 2 + 0.3 * torch.randn(n_samples, 1)

print(f"X shape: {X.shape}")
print(f"y shape: {y_true.shape}")


 ## Step 2: Visualize ข้อมูล

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X.numpy(), y_true.numpy(), alpha=0.6, label='Data')
plt.xlabel("X")
plt.ylabel("y")
plt.title("Linear Regression Data\n(True relationship: y = 3x + 2)")
plt.legend()
plt.grid(True)
plt.show()


 ## Step 3: กำหนด Parameters



 เราต้องหาค่า w และ b ที่ทำให้ `y = wx + b` ใกล้เคียงข้อมูลที่สุด

In [ ]:
# เริ่มต้นด้วยค่าสุ่ม
w = torch.randn(1, requires_grad=True)  # weight
b = torch.zeros(1, requires_grad=True)  # bias

print(f"ค่าเริ่มต้น:")
print(f"  w = {w.item():.4f} (target: 3.0)")
print(f"  b = {b.item():.4f} (target: 2.0)")


 ## Step 4: Training Loop



 **ขั้นตอนในแต่ละ epoch:**

 1. Forward pass: คำนวณ prediction

 2. คำนวณ loss (MSE)

 3. Backward pass: คำนวณ gradients

 4. Update parameters

 5. Clear gradients

In [ ]:
learning_rate = 0.1
n_epochs = 100
losses = []

for epoch in range(n_epochs):
    # 1. Forward pass
    y_pred = X * w + b
    
    # 2. คำนวณ MSE loss
    loss = ((y_pred - y_true) ** 2).mean()
    losses.append(loss.item())
    
    # 3. Backward pass
    loss.backward()
    
    # 4. Update parameters
    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
    
    # 5. Clear gradients
    w.grad.zero_()
    b.grad.zero_()
    
    # แสดงผลทุกๆ 20 epochs
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d}: Loss = {loss.item():.4f}, "
              f"w = {w.item():.4f}, b = {b.item():.4f}")

print(f"\n✅ Final Result:")
print(f"   w = {w.item():.4f} (target: 3.0)")
print(f"   b = {b.item():.4f} (target: 2.0)")


 ## Step 5: Visualize ผลลัพธ์

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
axes[0].plot(losses)
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("Loss Reduction During Training")
axes[0].grid(True)

# Prediction plot
with torch.no_grad():
    y_pred = X * w + b

axes[1].scatter(X.numpy(), y_true.numpy(), alpha=0.6, label="Actual Data")
axes[1].plot(X.numpy(), y_pred.numpy(), 'r-', linewidth=2, 
             label=f"Prediction: y = {w.item():.2f}x + {b.item():.2f}")
axes[1].set_xlabel("X")
axes[1].set_ylabel("y")
axes[1].set_title("Linear Regression Result")
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()


 ## ✏️ แบบฝึกหัด Lab A: Polynomial Regression



 **โจทย์:** ขยาย linear regression เป็น polynomial



 หาค่า a, b, c ที่ทำให้ `y = ax² + bx + c` ใกล้เคียงข้อมูล

In [ ]:
# สร้างข้อมูล polynomial
set_seed(42)
X_poly = torch.linspace(-2, 2, 100).unsqueeze(1)
y_poly_true = 0.5 * X_poly**2 - 1.5 * X_poly + 0.5 + 0.2 * torch.randn_like(X_poly)

# เขียนโค้ดของคุณที่นี่ (YOUR CODE HERE)
# 1. กำหนด parameters a, b, c
# 2. เขียน training loop
# 3. Plot ผลลัพธ์


 ---

 # 🧪 Lab B: MLP Classifier บน Toy Dataset

 ---



 มาสร้าง Neural Network Classifier ที่สมบูรณ์!

 ## Step 1: สร้างข้อมูล



 ใช้ dataset "moons" จาก scikit-learn

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

set_seed(42)

# สร้าง dataset รูปพระจันทร์เสี้ยว 2 อัน
X_np, y_np = make_moons(n_samples=500, noise=0.1)

# แบ่ง train/test (80%/20%)
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_np, y_np, test_size=0.2, random_state=42
)

# แปลงเป็น PyTorch tensors
X_train = torch.from_numpy(X_train_np).float()
X_test = torch.from_numpy(X_test_np).float()
y_train = torch.from_numpy(y_train_np).long()  # CrossEntropy ต้องการ long
y_test = torch.from_numpy(y_test_np).long()

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")


 ## Step 2: Visualize ข้อมูล

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Training data
axes[0].scatter(X_train_np[y_train_np == 0, 0], X_train_np[y_train_np == 0, 1], 
                c='blue', label='Class 0', alpha=0.7)
axes[0].scatter(X_train_np[y_train_np == 1, 0], X_train_np[y_train_np == 1, 1], 
                c='red', label='Class 1', alpha=0.7)
axes[0].set_title("Training Data")
axes[0].legend()
axes[0].grid(True)

# Test data
axes[1].scatter(X_test_np[y_test_np == 0, 0], X_test_np[y_test_np == 0, 1], 
                c='blue', label='Class 0', alpha=0.7)
axes[1].scatter(X_test_np[y_test_np == 1, 0], X_test_np[y_test_np == 1, 1], 
                c='red', label='Class 1', alpha=0.7)
axes[1].set_title("Test Data")
axes[1].legend()
axes[1].grid(True)

plt.suptitle("Moons Dataset", fontsize=14)
plt.tight_layout()
plt.show()


 ## Step 3: สร้าง MLP Classifier

In [ ]:
class MLPClassifier(nn.Module):
    """
    Multi-Layer Perceptron สำหรับ Binary Classification
    
    โครงสร้าง:
    Input(2) → Linear → ReLU → Linear → ReLU → Linear → Output(2)
    """
    
    def __init__(self, input_dim=2, hidden_dim=32, output_dim=2):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# สร้าง model
model = MLPClassifier(input_dim=2, hidden_dim=32, output_dim=2)
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")


 ## Step 4: กำหนด Loss และ Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


 ## Step 5: Training Loop

In [ ]:
n_epochs = 200
train_losses = []
test_accs = []

for epoch in range(n_epochs):
    # === Training ===
    model.train()
    
    # Forward
    logits = model(X_train)
    loss = criterion(logits, y_train)
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    train_losses.append(loss.item())
    
    # === Evaluation ===
    model.eval()
    with torch.no_grad():
        test_logits = model(X_test)
        test_preds = test_logits.argmax(dim=1)
        test_acc = (test_preds == y_test).float().mean()
        test_accs.append(test_acc.item())
    
    # แสดงผลทุกๆ 50 epochs
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1:3d}: Loss = {loss.item():.4f}, "
              f"Test Accuracy = {test_acc.item():.4f}")

print(f"\n✅ Final Test Accuracy: {test_accs[-1]:.2%}")


 ## Step 6: Visualize Training Progress

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
axes[0].plot(train_losses)
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("Training Loss")
axes[0].grid(True)

# Accuracy curve
axes[1].plot(test_accs)
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Test Accuracy")
axes[1].grid(True)

plt.tight_layout()
plt.show()


 ## Step 7: Visualize Decision Boundary

In [ ]:
def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    """
    แสดง decision boundary ของ classifier
    """
    model.eval()
    
    # สร้าง mesh grid
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    # Predict บน mesh
    grid = torch.from_numpy(np.c_[xx.ravel(), yy.ravel()]).float()
    with torch.no_grad():
        probs = torch.softmax(model(grid), dim=1)[:, 1].numpy()
    probs = probs.reshape(xx.shape)
    
    # Plot
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, probs, levels=50, cmap='RdBu', alpha=0.8)
    plt.colorbar(label='P(Class 1)')
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', edgecolors='white', 
                s=50, label='Class 0')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='red', edgecolors='white', 
                s=50, label='Class 1')
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_decision_boundary(model, X_test_np, y_test_np, 
                       title=f"Decision Boundary (Test Acc: {test_accs[-1]:.2%})")


 ## Step 8: บันทึกและโหลด Model



 **คำอธิบาย (ภาษาไทย):**



 การบันทึก model ที่ดีที่สุดคือบันทึก `state_dict` (parameters)

In [ ]:
# บันทึก model
model_path = "./mlp_classifier.pt"
torch.save(model.state_dict(), model_path)
print(f"✅ Model saved to: {model_path}")


In [ ]:
# โหลด model
loaded_model = MLPClassifier(input_dim=2, hidden_dim=32, output_dim=2)
loaded_model.load_state_dict(torch.load(model_path, weights_only=True))
loaded_model.eval()

# ทดสอบ
with torch.no_grad():
    test_logits = loaded_model(X_test)
    test_preds = test_logits.argmax(dim=1)
    test_acc = (test_preds == y_test).float().mean()
    
print(f"✅ Model loaded successfully - Test Accuracy: {test_acc:.4f}")


 ---

 # 📚 เฉลยแบบฝึกหัด

 ---

 ## เฉลย: แบบฝึกหัด 1.1 - Device Management

In [ ]:
def move_to_best_device_solution(tensor):
    """ย้าย tensor ไปยัง device ที่ดีที่สุด"""
    print(f"Device ปัจจุบัน: {tensor.device}")
    best_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    moved = tensor.to(best_device)
    print(f"ย้ายไปที่: {moved.device}")
    return moved

# ทดสอบ
print("ทดสอบฟังก์ชัน move_to_best_device:")
test_tensor = torch.ones(5)
result = move_to_best_device_solution(test_tensor)


 ## เฉลย: แบบฝึกหัด 2.1 - Tensor Manipulation

In [ ]:
matrix = torch.arange(16).reshape(4, 4).float()
print(f"Matrix:\n{matrix}")

# 1. Diagonal
diagonal = matrix.diag()
print(f"\n1. Diagonal: {diagonal}")

# 2. Upper-right 2x2
upper_right = matrix[:2, 2:]
print(f"\n2. Upper-right 2x2:\n{upper_right}")

# 3. Above mean
mean_val = matrix.mean()
above_mean = matrix[matrix > mean_val]
print(f"\n3. Mean = {mean_val:.2f}")
print(f"   Values above mean: {above_mean}")


 ## เฉลย: แบบฝึกหัด 2.2 - Softmax และ Cosine Similarity

In [ ]:
def softmax_solution(x, dim=-1):
    """คำนวณ softmax พร้อม numerical stability"""
    x_max = x.max(dim=dim, keepdim=True).values
    x_exp = torch.exp(x - x_max)  # ลบ max ป้องกัน overflow
    return x_exp / x_exp.sum(dim=dim, keepdim=True)

def batch_cosine_similarity_solution(a, b):
    """คำนวณ cosine similarity"""
    dot_product = (a * b).sum(dim=1)
    norm_a = torch.norm(a, dim=1)
    norm_b = torch.norm(b, dim=1)
    return dot_product / (norm_a * norm_b)

# ทดสอบ softmax
x = torch.tensor([[1., 2., 3.], [1., 1., 1.]])
result = softmax_solution(x)
print(f"Softmax result:\n{result}")
print(f"Row sums: {result.sum(dim=1)}")  # ควรเป็น 1

# ทดสอบ cosine similarity
a = torch.tensor([[1., 0., 0.], [1., 1., 0.]])
b = torch.tensor([[1., 0., 0.], [0., 1., 0.]])
sim = batch_cosine_similarity_solution(a, b)
print(f"\nCosine similarities: {sim}")  # [1.0, 0.707...]


 ## เฉลย: Polynomial Regression

In [ ]:
set_seed(42)
X_poly = torch.linspace(-2, 2, 100).unsqueeze(1)
y_poly_true = 0.5 * X_poly**2 - 1.5 * X_poly + 0.5 + 0.2 * torch.randn_like(X_poly)

# กำหนด parameters
a = torch.randn(1, requires_grad=True)
b_coef = torch.randn(1, requires_grad=True)
c = torch.randn(1, requires_grad=True)

# Training
learning_rate = 0.1
n_epochs = 500

for epoch in range(n_epochs):
    # Forward: y = ax² + bx + c
    y_pred = a * X_poly**2 + b_coef * X_poly + c
    loss = ((y_pred - y_poly_true) ** 2).mean()
    
    loss.backward()
    
    with torch.no_grad():
        a -= learning_rate * a.grad
        b_coef -= learning_rate * b_coef.grad
        c -= learning_rate * c.grad
    
    a.grad.zero_()
    b_coef.grad.zero_()
    c.grad.zero_()
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}: Loss={loss.item():.4f}")

print(f"\n✅ Result: y = {a.item():.2f}x^2 + {b_coef.item():.2f}x + {c.item():.2f}")
print(f"   Target: y = 0.50x^2 - 1.50x + 0.50")

# Plot
with torch.no_grad():
    y_pred = a * X_poly**2 + b_coef * X_poly + c

plt.figure(figsize=(10, 6))
plt.scatter(X_poly.numpy(), y_poly_true.numpy(), alpha=0.5, label='Data')
plt.plot(X_poly.numpy(), y_pred.numpy(), 'r-', linewidth=2, 
         label=f'Fit: {a.item():.2f}x^2 + {b_coef.item():.2f}x + {c.item():.2f}')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Polynomial Regression Result')
plt.legend()
plt.grid(True)
plt.show()


 ---

 # 🎯 สรุป Skills ที่ได้เรียนรู้

 ---



 ## ✅ เขียน Training Loop ได้

 ```python

 for epoch in range(n_epochs):

     model.train()

     outputs = model(X)

     loss = criterion(outputs, y)

     optimizer.zero_grad()  # Clear gradients

     loss.backward()        # Compute gradients

     optimizer.step()       # Update weights

 ```



 ## ✅ แก้ปัญหาที่พบบ่อย



 | Error | สาเหตุ | วิธีแก้ |

 |-------|--------|--------|

 | Size mismatch | มิติไม่ตรง | ตรวจ shape, ใช้ reshape |

 | Device mismatch | CPU/GPU ปนกัน | ย้ายไป device เดียวกัน |

 | Dtype mismatch | ชนิดข้อมูลไม่ตรง | ใช้ .float(), .long() |

 | Gradients เป็น None | ไม่ได้เรียก backward | ตรวจ requires_grad |

 | Gradients สะสม | ไม่ได้ clear | optimizer.zero_grad() |



 ## ✅ Save/Load Models

 ```python

 # Save

 torch.save(model.state_dict(), "model.pt")



 # Load

 model.load_state_dict(torch.load("model.pt", weights_only=True))

 model.eval()

 ```

 ---

 # 🚀 ขั้นตอนถัดไป

 ---



 **Day 2 Topics:**

 - Datasets และ DataLoaders

 - Transforms และ augmentation

 - Model architectures (CNNs, RNNs)

 - Training best practices

 - GPU optimization



 **แหล่งเรียนรู้เพิ่มเติม:**

 - [PyTorch Official Tutorials](https://pytorch.org/tutorials/)

 - [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)

 - [Deep Learning with PyTorch Book](https://pytorch.org/deep-learning-with-pytorch)

In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════════╗
║                                                                   ║
║  🎉 ยินดีด้วย! คุณเรียนจบ PyTorch Basics Lab แล้ว!                ║
║                                                                   ║
║  สิ่งที่ได้เรียนรู้:                                              ║
║  • Tensor คือพื้นฐานของ PyTorch                                   ║
║  • Autograd คำนวณ gradient อัตโนมัติ                               ║
║  • nn.Module คือ base class ของ neural networks                   ║
║  • Training loop: forward → loss → backward → step                ║
║                                                                   ║
╚═══════════════════════════════════════════════════════════════════╝
""")
